In [3]:
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import time
import math

# Definicje algorytmów
def dominates(p1, p2, criteria):
    better_in_all = True
    better_in_at_least_one = False
    for i in range(len(p1)):
        if criteria[i] == 'min':
            if p1[i] > p2[i]:
                better_in_all = False
                break
            elif p1[i] < p2[i]:
                better_in_at_least_one = True
        elif criteria[i] == 'max':
            if p1[i] < p2[i]:
                better_in_all = False
                break
            elif p1[i] > p2[i]:
                better_in_at_least_one = True
    return better_in_all and better_in_at_least_one

def naiveNonFilter(X, criteria):
    P = []
    number_of_comparisons = 0
    X_temp = X.copy()
    while len(X_temp) > 0:
        Y = X_temp[0]
        dominated = False
        to_remove = []
        for i in range(1, len(X_temp)):
            number_of_comparisons += 1
            if dominates(Y, X_temp[i], criteria):
                to_remove.append(X_temp[i])
            elif dominates(X_temp[i], Y, criteria):
                dominated = True
                to_remove.append(Y)
                Y = X_temp[i]
        if not dominated and Y not in P:
            P.append(Y)
        for item in to_remove:
            if item in X_temp:
                X_temp.remove(item)
        if Y in X_temp:
            X_temp.remove(Y)
    return P, number_of_comparisons

def naiveWithFilter(X, criteria):
    P = []
    number_of_comparisons = 0
    i = 0
    while i < len(X):
        Y = X[i]
        fl = 0
        j = i + 1
        while j < len(X):
            number_of_comparisons += 1
            if dominates(Y, X[j], criteria):
                X.pop(j)
            elif dominates(X[j], Y, criteria):
                Y = X.pop(j)
                X[i] = Y
                j = i + 1
            else:
                j += 1
        P.append(Y)
        X.pop(i)
    return P, number_of_comparisons

def distance(p1, p2):
    return math.sqrt(sum((p1[i] - p2[i]) ** 2 for i in range(len(p1))))

def naiveIdealPoint(X, criteria):
    # Znajdź punkt idealny na podstawie kryteriów
    ideal = []
    for i in range(len(X[0])):
        if criteria[i] == 'min':
            ideal_value = min(point[i] for point in X)
        else:
            ideal_value = max(point[i] for point in X)
        ideal.append(ideal_value)
    ideal = tuple(ideal)

    distances = [(distance(p, ideal), idx) for idx, p in enumerate(X)]
    distances.sort(key=lambda x: x[0])

    sorted_indices = [index for _, index in distances]

    P = []
    number_of_comparisons = 0
    m = 0
    X_temp = X.copy()

    while len(X_temp) > 1 and m < len(sorted_indices):
        current_p = X[sorted_indices[m]]
        if current_p in X_temp:
            P.append(current_p)
            X_temp.remove(current_p)

        dominated_p = []
        for p in X_temp:
            number_of_comparisons += 1
            if dominates(current_p, p, criteria):
                dominated_p.append(p)

        for dp in dominated_p:
            X_temp.remove(dp)

        m += 1
    if X_temp:
        P.append(X_temp[0])

    return P, number_of_comparisons

# Definicja aplikacji
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Znajdowanie Punktów Niezdominowanych")
        self.geometry("1000x700")
        self.create_widgets()

    def create_widgets(self):
        # Konfiguracja danych wejściowych
        input_frame = ttk.LabelFrame(self, text="Konfiguracja danych wejściowych")
        input_frame.pack(fill="x", padx=10, pady=5)

        ttk.Label(input_frame, text="Liczba punktów:").grid(row=0, column=0, padx=5, pady=5)
        self.num_points_var = tk.IntVar(value=50)
        ttk.Entry(input_frame, textvariable=self.num_points_var).grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(input_frame, text="Zakres wartości X (min, max):").grid(row=1, column=0, padx=5, pady=5)
        self.x_min_var = tk.DoubleVar(value=0)
        self.x_max_var = tk.DoubleVar(value=10)
        ttk.Entry(input_frame, textvariable=self.x_min_var, width=10).grid(row=1, column=1, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.x_max_var, width=10).grid(row=1, column=1, padx=5, pady=5, sticky="e")

        ttk.Label(input_frame, text="Zakres wartości Y (min, max):").grid(row=2, column=0, padx=5, pady=5)
        self.y_min_var = tk.DoubleVar(value=0)
        self.y_max_var = tk.DoubleVar(value=10)
        ttk.Entry(input_frame, textvariable=self.y_min_var, width=10).grid(row=2, column=1, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.y_max_var, width=10).grid(row=2, column=1, padx=5, pady=5, sticky="e")

        # Kryteria min/max
        ttk.Label(input_frame, text="Kryterium dla X:").grid(row=3, column=0, padx=5, pady=5)
        self.obj1_criteria = tk.StringVar(value="Minimize")
        ttk.OptionMenu(input_frame, self.obj1_criteria, "Minimize", "Minimize", "Maximize").grid(row=3, column=1, padx=5, pady=5)

        ttk.Label(input_frame, text="Kryterium dla Y:").grid(row=4, column=0, padx=5, pady=5)
        self.obj2_criteria = tk.StringVar(value="Minimize")
        ttk.OptionMenu(input_frame, self.obj2_criteria, "Minimize", "Minimize", "Maximize").grid(row=4, column=1, padx=5, pady=5)

        # Przyciski
        ttk.Button(input_frame, text="Generuj dane", command=self.generate_data).grid(row=5, column=0, padx=5, pady=5)
        ttk.Button(input_frame, text="Uruchom algorytmy", command=self.run_algorithms).grid(row=5, column=1, padx=5, pady=5)

        # Wybór algorytmów
        algo_frame = ttk.LabelFrame(self, text="Algorytmy")
        algo_frame.pack(fill="x", padx=10, pady=5)

        self.algo_vars = {}
        self.algo_names = ["Naive Non-Filter", "Naive With Filter", "Naive Ideal Point"]
        for i, name in enumerate(self.algo_names):
            var = tk.BooleanVar(value=True)
            self.algo_vars[name] = var
            ttk.Checkbutton(algo_frame, text=name, variable=var).grid(row=0, column=i, padx=5, pady=5)

        # Ramka wyników
        result_frame = ttk.Frame(self)
        result_frame.pack(fill="both", expand=True, padx=10, pady=5)

        self.result_text = tk.Text(result_frame, height=10)
        self.result_text.pack(fill="x", padx=5, pady=5)

        # Wykres
        self.figure = plt.Figure(figsize=(6, 5))
        self.ax = self.figure.add_subplot(111)
        self.canvas = FigureCanvasTkAgg(self.figure, master=result_frame)
        self.canvas.get_tk_widget().pack(fill="both", expand=True)

    def generate_data(self):
        num_points = self.num_points_var.get()
        x_min = self.x_min_var.get()
        x_max = self.x_max_var.get()
        y_min = self.y_min_var.get()
        y_max = self.y_max_var.get()

        self.X = []
        for _ in range(num_points):
            x = np.random.uniform(x_min, x_max)
            y = np.random.uniform(y_min, y_max)
            self.X.append((x, y))

        self.result_text.insert(tk.END, f"Wygenerowano {num_points} punktów.\n")
        self.plot_data()

    def run_algorithms(self):
        selected_algos = [name for name in self.algo_names if self.algo_vars[name].get()]
        if not selected_algos:
            messagebox.showwarning("Brak wybranego algorytmu", "Proszę wybrać co najmniej jeden algorytm.")
            return

        if not hasattr(self, 'X') or not self.X:
            messagebox.showwarning("Brak danych", "Proszę najpierw wygenerować dane.")
            return

        # Pobierz kryteria
        criteria = []
        if self.obj1_criteria.get() == "Minimize":
            criteria.append('min')
        else:
            criteria.append('max')

        if self.obj2_criteria.get() == "Minimize":
            criteria.append('min')
        else:
            criteria.append('max')

        # Wyczyszczenie poprzednich wyników
        self.result_text.delete(1.0, tk.END)
        self.ax.clear()

        colors = {'Naive Non-Filter': 'red', 'Naive With Filter': 'blue', 'Naive Ideal Point': 'green'}
        markers = {'Naive Non-Filter': 'o', 'Naive With Filter': '^', 'Naive Ideal Point': 's'}

        for algo_name in selected_algos:
            start_time = time.time()
            if algo_name == "Naive Non-Filter":
                P, no = naiveNonFilter(self.X.copy(), criteria)
            elif algo_name == "Naive With Filter":
                P, no = naiveWithFilter(self.X.copy(), criteria)
            elif algo_name == "Naive Ideal Point":
                P, no = naiveIdealPoint(self.X.copy(), criteria)
            else:
                continue
            end_time = time.time()
            elapsed_time = end_time - start_time

            self.result_text.insert(tk.END, f"{algo_name}:\n")
            self.result_text.insert(tk.END, f"Liczba porównań: {no}\n")
            self.result_text.insert(tk.END, f"Czas wykonania: {elapsed_time:.6f} sekund\n")
            self.result_text.insert(tk.END, f"Liczba punktów niezdominowanych: {len(P)}\n\n")

            # Wykres punktów niezdominowanych
            xs = [p[0] for p in P]
            ys = [p[1] for p in P]
            self.ax.scatter(xs, ys, color=colors.get(algo_name, 'black'), marker=markers.get(algo_name, 'o'), label=f"{algo_name}")

        # Wykres wszystkich punktów
        all_xs = [p[0] for p in self.X]
        all_ys = [p[1] for p in self.X]
        self.ax.scatter(all_xs, all_ys, color='gray', alpha=0.5, label='Wszystkie punkty')

        self.ax.legend()
        self.ax.set_xlabel('X')
        self.ax.set_ylabel('Y')
        self.canvas.draw()

    def plot_data(self):
        self.ax.clear()
        xs = [p[0] for p in self.X]
        ys = [p[1] for p in self.X]
        self.ax.scatter(xs, ys, color='gray', alpha=0.5, label='Wszystkie punkty')
        self.ax.set_xlabel('X')
        self.ax.set_ylabel('Y')
        self.canvas.draw()

if __name__ == "__main__":
    app = App()
    app.mainloop()
